## Alumno: 
## Curso y comision: Data Science 1950
## Tutor: Gustavo Bitocchi 

## Import libraries

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [28]:
# Modulos propios del proyecto que se encuentran en el directorio src
from src.utils.paths import data_dir


## Read Data

In [29]:
# Obtenemos la ruta de los csv's que vamos a utilizar
WINES_CSV_PATH = data_dir("intermediate", "df_wine_merged.csv")

WINES_CSV_PATH

WindowsPath('c:/Users/cvigo/data-projects/proyecto-final-vinos/vinos-corderhouse/data/intermediate/df_wine_merged.csv')

In [30]:
df_wines = pd.read_csv(WINES_CSV_PATH)

df_wines.head()

,winery,wine,year,rating,region,price,type,body,fixed_acidity,volatile_acidity,...,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,style
0,Vinedos de Paganos,La Nieta,2015,4.7,Rioja,82.58,Rioja Red,4.0,7.9,0.60,...,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,red
1,Emilio Moro,Malleolus de Valderramiro,2016,4.7,Ribera del Duero,84.95,Ribera Del Duero Red,5.0,7.5,0.50,...,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,red
2,Emilio Moro,Malleolus de Sanchomartin,2016,4.7,Ribera del Duero,116.54,Ribera Del Duero Red,5.0,6.7,0.58,...,1.8,0.097,15.0,65.0,0.9959,3.28,0.54,9.2,5,red
3,Bodegas Mauro,Terreus Paraje de Cueva Baja,2017,4.7,Castilla y Leon,66.07,Tempranillo,4.0,7.5,0.50,...,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,red
4,Vinedos de Paganos,La Nieta,2018,4.7,Rioja,89.90,Rioja Red,4.0,8.5,0.28,...,1.8,0.092,35.0,103.0,0.9969,3.30,0.75,10.5,7,red


## KNN

In [31]:
# Se toma como punto de referencia el promedio del año y de la edad 
price_Prom = df_wines['price'].mean()
year_Prom = df_wines['year'].mean()	
print(price_Prom,year_Prom)
Punto = {'price' : [price_Prom],
        'year': [year_Prom]}

36.90931702614465 2013.9277877518452


In [32]:
punto_nuevo = pd.DataFrame(Punto)
punto_nuevo

,price,year
0,36.909317,2013.927788


In [33]:
knn = KNeighborsClassifier(n_neighbors=3)
X = df_wines[['price', 'year']]
y = df_wines[['quality']]

knn.fit(X, y)

c:\Users\cvigo\.conda\envs\vinos_coderhouse\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [34]:
prediccion = knn.predict(punto_nuevo)
print(f"Se puede observar que el valor de quality predecido es {prediccion} en base a tres vecinos cercanos, es decir que para el punto observado la mayoria de vecinos tienen un quality igual 6")

Se puede observar que el valor de quality predecido es [6] en base a tres vecinos cercanos, es decir que para el punto observado la mayoria de vecinos tienen un quality igual 6


## Random Forest

In [35]:
# convertir el tipo de dato de la columna quality a categorical
df_wines.quality = df_wines.quality.astype('category')
# Retiramos las columnas que no van a ser parte de nuestro analisis
df_wines = df_wines.drop(['body', 'rating'], axis=1)

In [36]:
# Nuevo dataframe con solo las columnas de tipo numericas
df_wine_numeric = df_wines.select_dtypes(include=['float64', 'int64'])

In [37]:
df_wine_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5013 entries, 0 to 5012
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  5013 non-null   int64  
 1   price                 5013 non-null   float64
 2   fixed_acidity         5013 non-null   float64
 3   volatile_acidity      5013 non-null   float64
 4   citric_acid           5013 non-null   float64
 5   residual_sugar        5013 non-null   float64
 6   chlorides             5013 non-null   float64
 7   free_sulfur_dioxide   5013 non-null   float64
 8   total_sulfur_dioxide  5013 non-null   float64
 9   density               5013 non-null   float64
 10  pH                    5013 non-null   float64
 11  sulphates             5013 non-null   float64
 12  alcohol               5013 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 509.3 KB


**Insights**:

* El dataset se compone de 19 columnas de las cuales 18 corresponden a variables independientes, que usaremos para predecir el target.

* `quality` es la variable a predecir. Todas las variables no tiene valores nulos.

In [38]:
#Separamos en X e y
X = df_wine_numeric
y = df_wines["quality"]

In [39]:
#Separamos en train y test!
(X_train, X_test,
y_train, y_test) = train_test_split(X,
                                    y,
                                    stratify=y,
                                    test_size=0.30,
                                    random_state=11)

### Arbol de decision

In [40]:
#Creamos un arbol de decisión sencillo y lo fiteamos
tree = DecisionTreeClassifier(random_state=11)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

In [41]:
#Prediccion en Test
y_test_pred = tree.predict(X_test) 

In [47]:
#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'% de aciertos sobre el set de evaluación: {round(test_accuracy*100,3)}%')

% de aciertos sobre el set de evaluación: 68.684%


###  Random Forest

In [43]:
#Creamos un random forest!
model = RandomForestClassifier(random_state=11, n_estimators=200,
                               class_weight="balanced", max_features="log2")
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_features='log2',
                       n_estimators=200, random_state=11)

In [44]:
#Prediccion en Test
y_test_pred = model.predict(X_test) 

In [46]:
#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'% de aciertos sobre el set de evaluación: {round(test_accuracy*100,3)}%')

% de aciertos sobre el set de evaluación: 68.684%
